### **Simulação do Site de coleta**

O código a seguir simula do funcionamento do site de coleta de dados. Por questões de segurança relacionadas as chaves de acesso ao Google Cloud, essa versão foi criada para demonstrar seu funcionamento sem comprometer credenciais sensíveis. Ele mantém a lógica do código original, com as seguintes adaptações:
- Todo o código que faz referência ao Google Cloud foi comentado.
- O arquivo CSV é salvo localmente na pasta de execução do código.
- Para rodar o script, basta instalar as bibliotecas flask e pandas.

Após a execução do código, um link será gerado para acesso ao servidor local, semelhante a: http://127.0.0.1:5000
Lá podemos interagir e carregar os arquivo, podemos utilizar os próprios arquivos que temos nesse projeto em Arquivos/datacollect.

Após o upload, um arquivo CSV será gerado e salvo localmente. 
Na versão original, esse arquivo seria armazenado no bucket do Cloud Storage.

In [1]:
!pip install flask
!pip install pandas
!pip install openpyxl


In [ ]:
from flask import Flask, render_template, request
# from google.cloud import storage
import os
import pandas as pd
from io import BytesIO
import threading

# Definir a variável de ambiente no código
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "elite-mix-451813-f5-e45e608f8659.json"

# Criar o cliente do Cloud Storage
# client = storage.Client()

app = Flask(__name__)

# Configuração do bucket do Google Cloud Storage
# BUCKET_NAME = 'datacollect_mvp'

# Função para fazer upload do arquivo para o Google Cloud Storage
# def upload_to_gcs(file):
    # client = storage.Client()
    # bucket = client.get_bucket(BUCKET_NAME)

    # new_file, new_file_name = convert_xlsx_to_csv(file)
    
    # Definindo o caminho dentro do bucket (no caso, a "pasta" datacolect)
    # blob = bucket.blob(f'{new_file_name}')

    # blob.upload_from_file(new_file)
    # return blob.public_url

def convert_xlsx_to_csv(file):
        # Usa BytesIO para transformar a string binária em um objeto de arquivo
    df = pd.read_excel(file, sheet_name=None, header=None)
    
    sheet_name = list(df.keys())[0]  # Pega o nome da primeira planilha
    df_sheet = df[sheet_name]

    # Separar as linhas em duas partes: a primeira linha (informações do distribuidor)
    # e a segunda linha (informações das faturas)
    distribuidor_info = df_sheet.iloc[1, :5].values.tolist() # Linha 2 (informações do distribuidor)
    factura_info = df_sheet.iloc[3:, :10].values.tolist()  # Linha 4 (informações da fatura)
    factura_info = [item + [file.filename] for item in factura_info]
    # Criar um DataFrame para as informações do distribuidor
    distribuidor_df = pd.DataFrame([distribuidor_info], columns=["HEADER","DISTRIBUTOR", "NAME", "TAX", "DATA"])

    # Criar um DataFrame para as informações das faturas
    factura_df = pd.DataFrame(factura_info, columns=["IDENTIFICADOR","NUMERO DE FACTURA", 
                                                      "FECHA DE LA FACTURA", 
                                                      "TAX ID", 
                                                      "TIPO DE FACTURA", 
                                                      "CODIGO DE PRODUCTO", 
                                                      "NUMERO DE LOTE", 
                                                      "CANTIDAD FACTURADA", 
                                                      "UNIDAD DE MEDIDA", 
                                                      "PRECIO UNITARIO",
                                                      "FILE NAME"])

    # Adicionar as informações do distribuidor em cada linha de fatura
    factura_df["IDENTIFICADOR"] = distribuidor_df["HEADER"].iloc[0]
    factura_df["CODIGO DEL DISTRIBUIDOR"] = distribuidor_df["DISTRIBUTOR"].iloc[0]
    factura_df["NOMBRE DEL DISTRIBUIDOR"] = distribuidor_df["NAME"].iloc[0]
    factura_df["FECHA DEL ARCHIVO"] = distribuidor_df["DATA"].iloc[0]
    factura_df["TAX ID DIST"] = distribuidor_df["TAX"].iloc[0]

    # Nome do arquivo CSV que será salvo no bucket datachangecsv
    csv_file_name = file.filename.replace('.xlsx', '.csv')

    # Convertendo o DataFrame para CSV no formato de string
    csv_data = factura_df.to_csv(index=False, encoding="utf-8")

    new_file = BytesIO(csv_data.encode('utf-8'))

    print(f'Arquivo CSV {csv_file_name} criado com sucesso no bucket datachangecsv!')

    # Incluido para simulação
    # Define o caminho de salvamento no diretório padrão (onde o script está rodando)
    csv_file_name = file.filename.replace('.xlsx', '.csv')
    csv_file_path = os.path.join(os.getcwd(), csv_file_name)  # Obtém o diretório atual

    # Salva o DataFrame como CSV no diretório padrão
    factura_df.to_csv(csv_file_path, index=False, encoding="utf-8")

    print(f'Arquivo CSV salvo com sucesso em: {csv_file_path}')

    return new_file, csv_file_name


@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        file = request.files["file"]
        if file:
            # file_url = upload_to_gcs(file)
            file_url = convert_xlsx_to_csv(file)
            return render_template("confirmado.html")
    return render_template("index.html")

# if __name__ == "__main__":
#     app.run(debug=True)

def run_flask():
    app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5000)

# Rodar o Flask em uma thread separada
thread = threading.Thread(target=run_flask)
thread.start()

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.109:5000
Press CTRL+C to quit
127.0.0.1 - - [04/Apr/2025 08:12:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2025 08:12:23] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [04/Apr/2025 08:12:29] "POST / HTTP/1.1" 200 -


Arquivo CSV arquivo_template.csv criado com sucesso no bucket datachangecsv!
Arquivo CSV salvo com sucesso em: c:\Users\ramom\Desktop\Pós\Engenharia de dados\MVP--Criacao-Pipeline-de-Dados-para-Processamento-e-Analise-de-Sellout-no-GCP\Site\arquivo_template.csv


127.0.0.1 - - [04/Apr/2025 08:12:32] "GET / HTTP/1.1" 200 -
